# Preparation of the RWS data


Step 00 creates the combined tiff files from the individual ESRI-ascii files in the raw data.
0. Merge the ascii files for the area we want to analyse, save as a tiff

Step 01 is performed in the file CreatepointsXYZ.py using arcpy functionalities
1. Assign points to the lines and assign heights to those points (py 2.7 with arcgis script)

Step 02 is the last step in the datapreparation before the wavelet analysis can be performed
2. Check the heights on the lines


## 00 Merge the Ascii files into tiffs for the areas to be studied

In this notebook the Ascii files from the data as profided by RWS are merged into larger grids and saved as tiff files.
The tiff files then contain all the data within the areas MiddenWaal and TielstAndries.

The files are saved with the first and last measurment date in the dataset. As we are interested in median and low flow period, when dunes move between 2 (extreme low) to 5 (median+1000 m3 at Lobith) meters per day, and the measurements are taken mostly with one day difference, max 4 days difference.

- one last step to be performed is creating shape files indicating the area the different ascii files (still to be included in this notebook


In [1]:
import pandas as pd
import numpy as np
import os, glob

import re

from tqdm.notebook import tqdm

import rasterio
from rasterio.plot import show
from rasterio import crs
from rasterio.merge import merge


In [16]:
# rawDataFolder = r'c:\Users\lokinlr\Documents\01-DATA\RWS_multibeam data\ascii'
rawDataFolder = r'd:\multibeam\ZAAK_31024841 2011-2014'
outDataFolder = r'c:\Users\lokinlr\Documents\01-DATA\00_Dataprep'

# Coordinate system of the Tiffs is RD new
CRS = crs.CRS.from_proj4('+proj=sterea +lat_0=52.15616055555555 +lon_0=5.38763888888889 +k=0.9999079 +x_0=155000 +y_0=463000 +ellps=bessel +towgs84=565.417,50.3319,465.552,-0.398957,0.343988,-1.8774,4.0725 +units=m +no_defs')


For the section 'beween the bridges' aka 'MiddleWaal' the following 'vakken' are needed (from the first contractor):
- 03A
- 03B
- 03C
- 04A

or Vak3 all and Vak4 only A.

from the second contractor the following vakken are needed:
- 8
- 9
- 10
- 11


In [18]:
outDataDEMs = os.path.join('MiddenWaal','DEMS')
outDataMeta = os.path.join('MiddenWaal','Meta')
for dirs in tqdm(os.listdir(rawDataFolder)):
#     if not dirs == 'ROP_BRW_007':
#         continue
#     print(dirs)
    fileList = []
    ascList = []
    dateList = []
    rasterList = []

    if 'BM 194' in dirs:
        continue
#         for rt,dr,fl in os.walk(os.path.join(rawDataFolder,dirs)):
#             for file in fl:
#                 if (file[-4:] == '.asc') and (('Vak 08' in file) or 
#                                               ('Vak 09' in file) or 
#                                               ('Vak 10' in file) or 
#                                               ('Vak 11' in file)):
#                     ascList = np.append(ascList,os.path.join(rt,file))
#                     date = int(re.split(' |\.',file)[5])
#                     if date < 20000000:
#                         date = date+20000000
#                     dateList = np.append(dateList,date)
#                     rasterList = np.append(rasterList, rasterio.open(os.path.join(rt,file)))
#                     fileList = np.append(fileList,file)
                    
    elif 'ROP_BRW_020' in dirs:
        continue
#         files = os.listdir(os.path.join(rawDataFolder,dirs,'02 Grids 1x1m','BR2 cpl'))
#         for file in files:
#             if file == 'ROP_BRW_020_vak_BR2e_NAP_ESRI_20190802_CPL.asc': #File is XYZ format no esri asc file
#                 continue
#             ascList = np.append(ascList,
#                                 os.path.join(rawDataFolder,dirs,'02 Grids 1x1m','BR2 cpl',file))
#             date = int(re.split('_',file)[7])
#             if date < 2000000:
#                 date = date+20000000
#             dateList = np.append(dateList,date)
#             rasterList = np.append(rasterList,
#                                   rasterio.open(os.path.join(rawDataFolder,dirs,'02 Grids 1x1m','BR2 cpl',file)))
#             fileList = np.append(fileList,file)
            
    elif 'ROP_BRW_023' in dirs:
        continue
#         for rt,dr,fl in os.walk(os.path.join(rawDataFolder,dirs)):
#             for file in fl:
#                 if file[-4:] == '.asc' and (('vak08' in file) or 
#                                             ('vak09' in file) or
#                                             ('vak8' in file) or 
#                                             ('vak9' in file) or 
#                                             ('vak10' in file) or 
#                                             ('vak11' in file) or
#                                             ('Vak08' in file) or 
#                                             ('Vak09' in file) or
#                                             ('Vak8' in file) or 
#                                             ('Vak9' in file) or 
#                                             ('Vak10' in file) or 
#                                             ('Vak11' in file)):
#                     ascList = np.append(ascList,os.path.join(rt,file))
#                     rasterList = np.append(rasterList, rasterio.open(os.path.join(rt,file)))
#                     if 'NAP' in file:
#                         date = int(re.split('_',file)[6])
#                         if date < 20000000:
#                             date = date+20000000
#                         dateList = np.append(dateList,date)
#                     else:
#                         date = int(re.split('_',file)[5])
#                         if date < 20000000:
#                             date = date+20000000
#                         dateList = np.append(dateList,date)
#                     fileList = np.append(fileList,file)

    elif ('ROP' in dirs) or ('BHP 003' in dirs):
        for rt,dr,fl in os.walk(os.path.join(rawDataFolder,dirs)):
            for file in fl:
                if file == 'ROP_BRW_012_esri_vak11_NAP_20190415_Edo BR2f.asc':
                    continue
                if file == 'ROP_BRW_007_esri_vak10_NAP_20190204_Edo BR2e.asc':
                    continue
                if file == 'ROP_BRW_007_esri_vak11_NAP_20190204_Edo BR2f.asc':
                    continue   
                if file[-4:] == '.asc' and (('vak08' in file) or 
                                            ('vak09' in file) or
                                            ('vak8' in file) or 
                                            ('vak9' in file) or 
                                            ('vak10' in file) or 
                                            ('vak11' in file) or
                                            ('Vak08' in file) or 
                                            ('Vak09' in file) or
                                            ('Vak8' in file) or 
                                            ('Vak9' in file) or 
                                            ('Vak10' in file) or 
                                            ('Vak11' in file)):
                    if 'Niet geaccepteerd' in dirs:
                        date = int(re.split('_',file)[4])
                        if date < 20000000:
                            date = date+20000000
                        dateList = np.append(dateList,date)
                    elif len(re.split('_| ', file)) >=7:
                        date = int(re.split('_| ', file)[6])
                        if date < 20000000:
                            date = date+20000000
                        dateList = np.append(dateList,date)
                    if os.path.exists(os.path.join(outDataDEMs,f'raster_{dateList.min():8.0f}_{dateList.max():8.0f}.tif')):
                        continue
                    fileList = np.append(fileList,file)
                    ascList = np.append(ascList,os.path.join(rt,file))
                    rasterList = np.append(rasterList, rasterio.open(os.path.join(rt,file)))

    # look foor asc files with 03A 03B 03C or 04A (or 03 or 04), merge them and store the metadata in the meta file
    # create a dataframe with the files that are merged and the measuring dates of each file
    elif 'BM 002' in dirs:
        continue
        # vak 3 heeft een ander format (XYZ ipv esri-ascii grid)
    elif 'BM 005' in dirs:
        for rt,dr,fl in os.walk(os.path.join(rawDataFolder,dirs)):
            for file in fl:
                if file[-4:] == '.asc' and ((' 03 ' in file) or 
                                            (' 03A ' in file) or 
                                            (' 03B ' in file) or 
                                            (' 03C' in file) or 
                                            (' 04 ' in file) or 
                                            (' 04A ' in file)):
                    ascList = np.append(ascList,os.path.join(rt,file))
                    date = int(file.split(' ')[2])
                    if date < 20000000:
                        date = date+20000000
                    dateList = np.append(dateList,date)
                    rasterList = np.append(rasterList, rasterio.open(os.path.join(rt,file)))
                    fileList = np.append(fileList,file)
        if len(dateList) == 0:
            continue
        
        if os.path.exists(os.path.join(outDataDEMs,f'raster_{dateList.min():8.0f}_{dateList.max():8.0f}.tif')):
            continue
    
        meta = pd.DataFrame(columns = ['FileName','FileLoc','Date'])
        meta['FileName'] = fileList
        meta['FileLoc'] = ascList
        meta['Date'] = dateList

        meta.to_csv(os.path.join(outDataMeta,
                                f'meta_{dateList.min():8.0f}_{dateList.max():8.0f}.csv'))

        mosaic, out_trans = merge(rasterList)

        with rasterio.open(os.path.join(outDataDEMs,f'raster_{dateList.min():8.0f}_{dateList.max():8.0f}.tif'),
                           'w',
                           driver = 'GTiff',
                           height = mosaic.shape[1],
                           width = mosaic.shape[2],
                           count = 1,
                           dtype = rasterio.float32,
                           crs = CRS,
                           transform = out_trans,
                           nodata = rasterList[0].nodata #9999.0 voor 1e deel -9999.0 voor 2e deel
                          ) as dst:
            dst.write(mosaic)   
        # pakt vak 3 niet
    
    else:
        continue
        for rt,dr,fl in os.walk(os.path.join(rawDataFolder,dirs)):
            for file in fl:
                if file[-4:] == '.asc' and ((' 03 ' in file) or 
                                            (' 03A ' in file) or 
                                            (' 03B ' in file) or 
                                            (' 03C' in file) or 
                                            (' 04 ' in file) or 
                                            (' 04A ' in file)):
                    ascList = np.append(ascList,os.path.join(rt,file))
                    date = int(file.split(' ')[2])
                    if date < 20000000:
                        date = date+20000000
                    dateList = np.append(dateList,date)
                    rasterList = np.append(rasterList, rasterio.open(os.path.join(rt,file)))
                    fileList = np.append(fileList,file)
        if len(dateList) == 0:
            continue
        
        if os.path.exists(os.path.join(outDataDEMs,f'raster_{dateList.min():8.0f}_{dateList.max():8.0f}.tif')):
            continue
    
        meta = pd.DataFrame(columns = ['FileName','FileLoc','Date'])
        meta['FileName'] = fileList
        meta['FileLoc'] = ascList
        meta['Date'] = dateList

        meta.to_csv(os.path.join(outDataMeta,
                                f'meta_{dateList.min():8.0f}_{dateList.max():8.0f}.csv'))

        mosaic, out_trans = merge(rasterList)

        with rasterio.open(os.path.join(outDataDEMs,f'raster_{dateList.min():8.0f}_{dateList.max():8.0f}.tif'),
                           'w',
                           driver = 'GTiff',
                           height = mosaic.shape[1],
                           width = mosaic.shape[2],
                           count = 1,
                           dtype = rasterio.float32,
                           crs = CRS,
                           transform = out_trans,
                           nodata = rasterList[0].nodata #9999.0 voor 1e deel -9999.0 voor 2e deel
                          ) as dst:
            dst.write(mosaic)    
    
    

  0%|          | 0/15 [00:00<?, ?it/s]

For the section 'Tiel - St.Andrie' aka 'LTD section' the following 'vakken' are needed (from the first contractor):
-   04B
-   04C
-   05A

or Vak3 all and Vak4 only A r

from the second contractor the following vakken are needed:

- 12
- 13
- 14


In [10]:
dateList

array([20140717., 20140717., 20140718., 20140718.])

In [ ]:
outDataDEMs = os.path.join(outDataFolder,'TielstAndries','DEMS')
outDataMeta = os.path.join(outDataFolder,'TielstAndries','Meta')
for dirs in tqdm(os.listdir(rawDataFolder)):
#     if not dirs == 'ROP_BRW_013':
#         continue
#     print(dirs)
    fileList = []
    ascList = []
    dateList = []
    rasterList = []

    if 'BM 194' in dirs:
        for rt,dr,fl in os.walk(os.path.join(rawDataFolder,dirs)):
            for file in fl:
                if (file[-4:] == '.asc') and (('Vak 12' in file) or 
                                              ('Vak 13' in file) or 
                                              ('Vak 14' in file)):
                    ascList = np.append(ascList,os.path.join(rt,file))
                    date = int(re.split(' |\.',file)[5])
                    if date < 20000000:
                        date = date+20000000
                    dateList = np.append(dateList,date)
                    rasterList = np.append(rasterList, rasterio.open(os.path.join(rt,file)))
                    fileList = np.append(fileList,file)
                    
    elif 'ROP_BRW_020' in dirs:
        files = os.listdir(os.path.join(rawDataFolder,dirs,'02 Grids 1x1m','BR3 cpl'))
        for file in files:
            ascList = np.append(ascList,
                                os.path.join(rawDataFolder,dirs,'02 Grids 1x1m','BR3 cpl',file))
            date = int(re.split('_',file)[7])
            if date < 2000000:
                date = date+20000000
            dateList = np.append(dateList,date)
            rasterList = np.append(rasterList,
                                  rasterio.open(os.path.join(rawDataFolder,dirs,'02 Grids 1x1m','BR3 cpl',file)))
            fileList = np.append(fileList,file)

    elif ('ROP' in dirs) or ('BHP 003' in dirs):
        for rt,dr,fl in os.walk(os.path.join(rawDataFolder,dirs)):
            for file in fl:
                if file == 'ROP_BRW_033_vak14_NAP_20200203_pien BR3c.asc':
                    continue
                if file == 'ROP_BRW_009_esri_vak14_NAP_20190227_Edo BR3c.asc': #weird values and extreme high (not valid)
                    continue
                if file == 'ROP_BRW_013_esri_vak13_NAP_20190502_Edo BR3b.asc':
                    continue
                
                if file[-4:] == '.asc' and (('vak12' in file) or 
                                            ('vak13' in file) or 
                                            ('vak14' in file) or
                                            ('Vak12' in file) or 
                                            ('Vak13' in file) or
                                            ('Vak14' in file)):
                    ascList = np.append(ascList,os.path.join(rt,file))
                    rasterList = np.append(rasterList, rasterio.open(os.path.join(rt,file)))
                    if 'Niet geaccepteerd' in dirs:
                        date = int(re.split('_',file)[4])
                        if date < 20000000:
                            date = date+20000000
                        dateList = np.append(dateList,date)
                    elif len(re.split('_| ', file)) >=7:
                        date = int(re.split('_| ', file)[6])
                        if date < 20000000:
                            date = date+20000000
                        dateList = np.append(dateList,date)
                    fileList = np.append(fileList,file)

    # look foor asc files with 03A 03B 03C or 04A (or 03 or 04), merge them and store the metadata in the meta file
    # create a dataframe with the files that are merged and the measuring dates of each file
    else:
        for rt,dr,fl in os.walk(os.path.join(rawDataFolder,dirs)):
            for file in fl:
                if file[-4:] == '.asc' and ((' 04 ' in file) or 
                                            (' 04B ' in file) or 
                                            (' 04C ' in file) or 
                                            (' 05 ' in file) or 
                                            (' 05A ' in file)):
                    ascList = np.append(ascList,os.path.join(rt,file))
                    date = int(file.split(' ')[2])
                    if date < 20000000:
                        date = date+20000000
                    dateList = np.append(dateList,date)
                    rasterList = np.append(rasterList, rasterio.open(os.path.join(rt,file)))
                    fileList = np.append(fileList,file)
#     print(fileList)
#     print(dateList)
    meta = pd.DataFrame(columns = ['FileName','FileLoc','Date'])
    meta['FileName'] = fileList
    meta['FileLoc'] = ascList
    meta['Date'] = dateList
        
    meta.to_csv(os.path.join(outDataMeta,
                            f'meta_{dateList.min():8.0f}_{dateList.max():8.0f}.csv'))
    
    mosaic, out_trans = merge(rasterList)
    
    with rasterio.open(os.path.join(outDataDEMs,f'raster_{dateList.min():8.0f}_{dateList.max():8.0f}.tif'),
                       'w',
                       driver = 'GTiff',
                       height = mosaic.shape[1],
                       width = mosaic.shape[2],
                       count = 1,
                       dtype = rasterio.float32,
                       crs = CRS,
                       transform = out_trans,
                       nodata = rasterList[0].nodata #9999.0
                      ) as dst:
        dst.write(mosaic)    
    
   